### Importing libraries

In [1]:
!pip install ipynb

In [2]:
import logging
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
import numpy as np
import time
import os
import sys
from ipynb.fs.defs.SiameseModel import Recognizer
import logging
import tensorflow as tf

### Image Dimensions

In [3]:
data_dimension = 128

### Input and target vectors

In [4]:
# Loading input and target vectors

X1 = np.load('x1.npy')
X2 = np.load('x2.npy')
Y = np.load('y.npy')

# Reshaping the target vectors
X1 = X1.reshape( ( X1.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )
X2 = X2.reshape( ( X2.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )

print( X1.shape )
print( X2.shape )
print( Y.shape )

(625, 49152)
(625, 49152)
(625,)


### Training and saving the model

In [5]:
# Creating an instance of siamese model
recognizer = Recognizer()

In [6]:
# Defining hyper parameters
parameters = {
    'batch_size' : 6 ,
    'epochs' : 5 ,
    'callbacks' : None , # [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
    'val_data' : None
}

# Training the siamese model
recognizer.fit( [ X1 , X2 ], Y, hyperparameters=parameters)

105/105 [==============================] - 280s 3s/step - loss: 0.4002
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 49152)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 49152)]      0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 64)           54243072    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda) 

In [7]:
# saving the model
recognizer.save_model('model.h5')

### Loading images from directory

In [8]:
# Enter the path name untill 'images' [ Actual images are stored at ...images/sanyam/image0.png]
# Here enter the path till the local images
# Custom images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\Test_Images\
# p1 : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Chetana\
# p2 : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Prabhleen\
# p3 : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Sanyam\
# p4 :  C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Taruneesh\
# p5 :  C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Neha\

dir_path_custom_images = input( 'Enter images directory path for custom images : ')
dir_path_class1_images = input( 'Enter images directory path for first class images : ')
dir_path_class2_images = input( 'Enter images directory path for second class images : ')
dir_path_class3_images = input( 'Enter images directory path for third class images : ')
dir_path_class4_images = input( 'Enter images directory path for fourth class images : ')
dir_path_class5_images = input( 'Enter images directory path for fifth class images : ')




Enter images directory path for custom images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\Test_Images\
Enter images directory path for first class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Chetana\
Enter images directory path for second class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Prabhleen\
Enter images directory path for third class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Sanyam\
Enter images directory path for fourth class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Taruneesh\
Enter images directory path for fifth class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\ima

In [9]:
# storing the numpy arrays of all images
custom_images = recognizer.prepare_images_from_dir( dir_path_custom_images  )
class_1_images = recognizer.prepare_images_from_dir( dir_path_class1_images )
class_2_images = recognizer.prepare_images_from_dir( dir_path_class2_images )
class_3_images = recognizer.prepare_images_from_dir( dir_path_class3_images )
class_4_images = recognizer.prepare_images_from_dir( dir_path_class4_images )
class_5_images = recognizer.prepare_images_from_dir( dir_path_class5_images )

In [10]:
# printing shape of all the images
print("The shape of the numpy array of class first images", class_1_images.shape)
print("The shape of the numpy array of class second images", class_2_images.shape)
print("The shape of the numpy array of class third images", class_3_images.shape)
print("The shape of the numpy array of class fourth images", class_4_images.shape)
print("The shape of the numpy array of class fifth images", class_5_images.shape)


print("The shape of the numpy array of custom images", custom_images.shape)

The shape of the numpy array of class first images (5, 49152)
The shape of the numpy array of class second images (5, 49152)
The shape of the numpy array of class third images (5, 49152)
The shape of the numpy array of class fourth images (5, 49152)
The shape of the numpy array of class fifth images (5, 49152)
The shape of the numpy array of custom images (10, 49152)


### Classifying images

In [11]:
# Predicting the score for each image

# scores stores a list of scores for each custom image with the train dataset
scores = list()

# labels is a list of labels for each custom image against the class label of trained image
labels = list()

# looping through all custom images
for image in custom_images:
    label = list()
    score = list()
    # looping across images of class 1
    for sample in class_1_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 0 to this class
        label.append( 0 )
    # looping across images of class 2
    for sample in class_2_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 1 for this class
        label.append( 1 )
    # looping across images of class 3
    for sample in class_3_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 1 for this class
        label.append( 2 )
    # looping across images of class 4
    for sample in class_4_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 1 for this class
        label.append( 3 )
    # looping across images of class 5
    for sample in class_5_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 1 for this class
        label.append( 4 )
    labels.append( label )
    scores.append( score )

In [16]:
print(len(labels[0]))

25


### Analysing the scores and labels

In [12]:
print("The scores are: ", scores)
print("The labels are: ", labels)

scores = np.array( scores )
labels = np.array( labels )

print('The shape of the numpy array of labels are: ', scores.shape)
print('The shape of the numpy array of scores are:', labels.shape )
print('The scores are: ', scores)
print('The lables are: ', labels)

The scores are:  [[array([0.37769634], dtype=float32), array([0.3199545], dtype=float32), array([0.2945462], dtype=float32), array([0.37769634], dtype=float32), array([0.37769634], dtype=float32), array([0.39993095], dtype=float32), array([0.33492082], dtype=float32), array([0.4293275], dtype=float32), array([0.3586761], dtype=float32), array([0.47457254], dtype=float32), array([0.26112923], dtype=float32), array([0.295502], dtype=float32), array([0.35696745], dtype=float32), array([0.23559189], dtype=float32), array([0.35696745], dtype=float32), array([0.04436329], dtype=float32), array([0.04671681], dtype=float32), array([0.04436323], dtype=float32), array([0.04671681], dtype=float32), array([0.04436323], dtype=float32), array([0.14536199], dtype=float32), array([0.11951816], dtype=float32), array([0.11203879], dtype=float32), array([0.19507438], dtype=float32), array([0.08503631], dtype=float32)], [array([0.2886338], dtype=float32), array([0.23927256], dtype=float32), array([0.25376

### Classifying into classes based on best score

In [13]:
for i in range( custom_images.shape[0] ) :
    index = np.argmax( scores[i] )
    label_ = labels[i][index]
    print( 'IMAGE {} is {} with confidence of {}'.format( i+1  , label_ , scores[i][index][0] ) )

IMAGE 1 is 1 with confidence of 0.4745725393295288
IMAGE 2 is 1 with confidence of 0.49923449754714966
IMAGE 3 is 1 with confidence of 0.4350891411304474
IMAGE 4 is 3 with confidence of 0.49920007586479187
IMAGE 5 is 1 with confidence of 0.4010492265224457
IMAGE 6 is 1 with confidence of 0.4296397566795349
IMAGE 7 is 4 with confidence of 0.3761540651321411
IMAGE 8 is 2 with confidence of 0.43531787395477295
IMAGE 9 is 3 with confidence of 0.4150741398334503
IMAGE 10 is 3 with confidence of 0.49923449754714966
